In [1]:
import torch

In [2]:
from torch import nn
from torch.utils.data import DataLoader

In [3]:
from torchvision import datasets

In [4]:
from torchvision.transforms import ToTensor

In [5]:
training_data = datasets.FashionMNIST(root='data',train=True,download=True,transform=ToTensor())

In [6]:
test_data = datasets.FashionMNIST(root='data',train=False,download=True,transform=ToTensor())

In [7]:
batch_size = 64
train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)
for X, y in test_dataloader:
    print(f"Shape of X: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break
    

Shape of X: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [8]:
device = ("cuda" if torch.cuda.is_available()
          else "mps"
          if torch.backends.mps.is_available()
          else "cpu")
print(f"Using {device} device")
          

Using mps device


In [9]:
#Defining Model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
#Optimising Model Parameters
# To train a model we need an optimizer and a loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred,y)
        # Back propagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch%100==0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f"Loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [12]:
#Checking models performance against testing data
def test(dataloader, mode, loss_fn):
    size = len(dataloader.dataset)
    num_batches= len(dataloader)
    model.eval()
    test_loss,correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /=num_batches
    correct /=size
    print(f"Test Error: \nAccuracy: {(100*correct):>0.1f}%, Avg Loss: {test_loss:>8f} \n")

In [13]:
epochs = 5
for t in range(epochs):
    print(f"Epochs {t+1}\n------------")
    train(train_dataloader,model,loss_fn,optimizer)
    test(test_dataloader,model,loss_fn)
print("Done!!")

Epochs 1
------------
Loss: 2.307648 [   64/60000]
Loss: 2.289631 [ 6464/60000]
Loss: 2.271587 [12864/60000]
Loss: 2.259353 [19264/60000]
Loss: 2.243279 [25664/60000]
Loss: 2.215984 [32064/60000]
Loss: 2.225640 [38464/60000]
Loss: 2.194904 [44864/60000]
Loss: 2.187423 [51264/60000]
Loss: 2.160079 [57664/60000]
Test Error: 
Accuracy: 41.6%, Avg Loss: 2.149337 

Epochs 2
------------
Loss: 2.163117 [   64/60000]
Loss: 2.149502 [ 6464/60000]
Loss: 2.095654 [12864/60000]
Loss: 2.103272 [19264/60000]
Loss: 2.053127 [25664/60000]
Loss: 2.002829 [32064/60000]
Loss: 2.025206 [38464/60000]
Loss: 1.956911 [44864/60000]
Loss: 1.952277 [51264/60000]
Loss: 1.884437 [57664/60000]
Test Error: 
Accuracy: 57.6%, Avg Loss: 1.880925 

Epochs 3
------------
Loss: 1.915043 [   64/60000]
Loss: 1.883374 [ 6464/60000]
Loss: 1.773471 [12864/60000]
Loss: 1.800348 [19264/60000]
Loss: 1.688842 [25664/60000]
Loss: 1.662242 [32064/60000]
Loss: 1.664888 [38464/60000]
Loss: 1.587520 [44864/60000]
Loss: 1.602051 [5126

In [14]:
torch.save(model.state_dict(),"model.pth")
print("Model Saved")

Model Saved


In [15]:
#Loading Model
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth",weights_only=True))

<All keys matched successfully>

In [19]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
model.eval()
x,y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x=x.to(device)
    pred = model(x)
    prediction, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted:"{prediction}", Actual: "{actual}"')

Predicted:"Ankle boot", Actual: "Ankle boot"
